# Step 1 | Platform Setup

## Step 1.1 | Check Environment

In [1]:
!where python

C:\Anaconda\envs\tf-gpu\python.exe


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3050 4GB Laptop GPU


## Step 1.2 | Import Libraries

In [3]:
# !pip install pandas numpy matplotlib seaborn scikit-learn
# !pip install lightgbm xgboost tqdm
# !pip install pyarrow
# !pip install multiprocess

In [4]:
import pandas as pd
import numpy as np

# Step 2 | Load Data

## Step 2.1 | Load CSV Files

### Step 2.1.1 | Sales Data

In [5]:
# Load main training data (wide format: 1 row per item, 1 column per day)

sales = pd.read_csv('sales_train_evaluation.csv')
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


### Step 2.1.2 | Calendar Data

In [6]:
calendar = pd.read_csv('calendar.csv')
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


### Step 2.1.3 | Price Data

In [7]:
sell_prices = pd.read_csv('sell_prices.csv')
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


## Step 2.2 | Convert Sales to Long Format

In [8]:
# Melt the sales dataframe
sales_long = pd.melt(
    sales, 
    id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
    var_name='d', 
    value_name='sales'
)

sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


## Step 2.3 | Merge Data

### Step 2.3.1 | Merge Calendar Data

In [9]:
sales_long = sales_long.merge(calendar, on='d', how='left')
sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,NaN,NaN,NaN,NaN,0,0,0


### Step 2.3.2 | Merge Price Data

In [10]:
sales_long = sales_long.merge(sell_prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN


## Step 2.4 | Sort Sales Data

In [11]:
sales_long.sort_values(by=['id', 'date'], inplace=True)

# Step 3 | Memory Optimization

## Step 3.1 | Review Current Memory Usage

In [12]:
# Optional: See memory usage before optimization

print("Initial shape:", sales_long.shape)
print("Initial memory usage (MB):", sales_long.memory_usage(deep=True).sum() / 1024**2)

# Preview columns
sales_long.columns.tolist()

Initial shape: (59181090, 22)
Initial memory usage (MB): 45849.675998687744


['id',
 'item_id',
 'dept_id',
 'cat_id',
 'store_id',
 'state_id',
 'd',
 'sales',
 'date',
 'wm_yr_wk',
 'weekday',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI',
 'sell_price']

## Step 3.2 | Drop Redundant Columns

In [13]:
cols_to_drop = [
    'wday', 'wm_yr_wk',
    'event_name_2', 'event_type_2',
]

sales_long.drop(columns=[col for col in cols_to_drop if col in sales_long.columns], inplace=True)

## Step 3.3 | Downcast Numeric Columns

In [14]:
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif col_type == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

sales_long = reduce_mem_usage(sales_long)

## Step 3.4 | Convert Object Columns to Categorical

In [15]:
for col in sales_long.select_dtypes(include='object').columns:
    sales_long[col] = sales_long[col].astype('category')

## Step 3.5 | Convert "d" Column to Numeric

In [16]:
sales_long['d_num'] = sales_long['d'].str.replace('d_', '').astype('int16')

In [17]:
sales_long.drop(columns=['d'], inplace=True)

## Step 3.6 | Review New Memory Usage

In [18]:
print(sales_long.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Index: 59181090 entries, 1612 to 59179652
Data columns (total 18 columns):
 #   Column        Dtype   
---  ------        -----   
 0   id            category
 1   item_id       category
 2   dept_id       category
 3   cat_id        category
 4   store_id      category
 5   state_id      category
 6   sales         int16   
 7   date          category
 8   weekday       category
 9   month         int8    
 10  year          int16   
 11  event_name_1  category
 12  event_type_1  category
 13  snap_CA       int8    
 14  snap_TX       int8    
 15  snap_WI       int8    
 16  sell_price    float32 
 17  d_num         int16   
dtypes: category(10), float32(1), int16(3), int8(4)
memory usage: 1.9 GB
None


In [19]:
print("Optimized shape:", sales_long.shape)
print("Optimized memory usage (MB):", sales_long.memory_usage(deep=True).sum() / 1024**2)

Optimized shape: (59181090, 18)
Optimized memory usage (MB): 1979.3422060012817


## Step 3.7 | Remove Unncessary Storage

In [20]:
import gc
gc.collect()

0

# Step 4: Export New Pandas DataFrame to External File

In [21]:
sales_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,sales,date,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,sell_price,d_num
1612,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,2011-01-29,Saturday,1,2011,NaN,NaN,0,0,0,2.0,1
32102,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,0,2011-01-30,Sunday,1,2011,NaN,NaN,0,0,0,2.0,2
62592,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,0,2011-01-31,Monday,1,2011,NaN,NaN,0,0,0,2.0,3
93082,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,2011-02-01,Tuesday,2,2011,NaN,NaN,1,1,0,2.0,4
123572,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,4,2011-02-02,Wednesday,2,2011,NaN,NaN,1,0,1,2.0,5


In [22]:
# Export to Parquet file

sales_long.to_parquet('Step1_LongDF_LSTM.parquet', index=False)